### 2-1

In [ ]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# .env 파일 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Groq 모델 설정 (이전 노트북 파일에서 사용된 설정 재활용)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5
)

# PromptTemplate 정의
prompt = PromptTemplate.from_template(
    """당신은 사용자가 질문한 내용을 기반으로 유용한 정보를 제공하는 친절한 챗봇입니다.
    질문: {question}
    이 질문에 대해 자세하고 유익한 답변을 제공해주세요."""
)

# RunnableParallel을 사용하여 'question' 인풋을 처리
# 'question' 키의 값을 그대로 전달 (RunnablePassthrough)
# 그 값을 prompt에 연결하고, prompt 결과를 model에 연결
chain = (
    {"question": RunnablePassthrough()}
    | prompt
    | model
)

# 인보커를 사용하여 질문에 대한 응답 받기
question_input = "LangChain이란 무엇인가요?"
response = chain.invoke(question_input)

print(f"질문: {question_input}")
print(f"답변: {response.content}")

### 2-2

In [ ]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# .env 파일 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Groq 모델 설정 (이전 노트북 파일에서 사용된 설정 재활용)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5
)

# PromptTemplate 정의 (ChatPromptTemplate 사용)
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 사용자에게 유용한 정보를 제공하는 AI 어시스턴트입니다."),
    ("user", "{question}")
])

# 체인 정의: prompt -> model -> output_parser
# | 연산자를 사용하여 구성 요소를 연결합니다.
chain = prompt | model | StrOutputParser()

# 체인 실행
question_input = "대한민국의 수도는 어디인가요?"
response_content = chain.invoke({"question": question_input})

print(f"질문: {question_input}")
print(f"답변: {response_content}")

### 2-3

In [ ]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field # Pydantic 모델 정의를 위해 임포트

# .env 파일 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Groq 모델 설정 (이전 노트북 파일에서 사용된 설정 재활용)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5
)

# JsonOutputParser에 사용할 Pydantic 모델 정의
# LLM이 생성할 JSON 응답의 스키마를 정의합니다.
class Person(BaseModel):
    name: str = Field(description="사람의 이름")
    age: int = Field(description="사람의 나이")
    city: str = Field(description="사람이 사는 도시")

# JsonOutputParser 초기화, Pydantic 모델 전달
parser = JsonOutputParser(pydantic_object=Person)

# PromptTemplate 정의
# LLM에게 JSON 형식으로 응답하도록 지시하고, 파서의 형식 지침을 포함합니다.
# 주의: 이 프롬프트는 LLM에게 명확하게 JSON 출력을 요청해야 합니다.
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 사용자의 정보를 기반으로 JSON 형식의 정보를 제공하는 어시스턴트입니다.\n{format_instructions}"),
    ("user", "이름이 {name}이고 나이가 {age}인 사람의 정보를 JSON으로 알려주세요. 사는 도시는 서울입니다.") # 질문 수정: JSON 출력 명시
])

# 체인 정의: prompt -> model -> parser
chain = prompt | model | parser

# 체인 실행
# LLM이 예상하는 JSON 스키마와 일치하도록 입력을 제공합니다.
input_data = {"name": "김철수", "age": 30, "format_instructions": parser.get_format_instructions()}
parsed_output = chain.invoke(input_data)

print(f"입력 데이터: {input_data}")
print(f"파싱된 출력 (Type: {type(parsed_output)}): {parsed_output}")
# 파싱된 출력에서 개별 필드에 접근 가능
# 딕셔너리 접근 방식으로 수정
print(f"이름: {parsed_output['name']}, 나이: {parsed_output['age']}, 도시: {parsed_output['city']}")

### 2-4

In [ ]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# .env 파일 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Groq 모델 설정 (이전 노트북 파일에서 사용된 설정 재활용)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5
)

# 첫 번째 체인: 날씨 정보 요청
weather_prompt = PromptTemplate.from_template(
    "오늘 {city}의 날씨는 어떤가요? 날씨 정보를 요약해서 알려주세요."
)
weather_chain = {"city": RunnablePassthrough()} | weather_prompt | model | StrOutputParser()

# 두 번째 체인: 뉴스 요약 요청
news_prompt = PromptTemplate.from_template(
    "오늘의 주요 {category} 뉴스에 대해 요약해서 알려주세요. 3문장 이내로."
)
news_chain = {"category": RunnablePassthrough()} | news_prompt | model | StrOutputParser()

# RunnableParallel을 사용하여 두 체인을 병렬로 실행
# 각 체인에 필요한 인풋을 정의합니다.
full_chain = RunnableParallel(
    current_weather=weather_chain,
    today_news=news_chain
)

# 체인 실행 (두 체인 모두에 필요한 인풋 제공)
# 'city'는 weather_chain의 input으로, 'category'는 news_chain의 input으로 들어갑니다.
result = full_chain.invoke({
    "city": "서울",
    "category": "경제"
})

print("==== 병렬 처리 결과 ====")
print(f"서울 날씨: {result['current_weather']}")
print(f"오늘의 경제 뉴스: {result['today_news']}")

# 다른 예시: 병렬 처리되는 인풋의 키가 다를 경우
# RunnableParallel은 키를 매핑해주므로 아래와 같이 정의할 수 있습니다.
another_full_chain = RunnableParallel(
    seoul_weather=weather_chain.bind(city="서울"), # city 인풋을 고정
    tech_news=news_chain.bind(category="기술")    # category 인풋을 고정
)

# 이 경우 invoke()에 인풋을 전달할 필요가 없습니다.
result_fixed_inputs = another_full_chain.invoke({})

print("\n==== 고정된 인풋 병렬 처리 결과 ====")
print(f"서울 날씨: {result_fixed_inputs['seoul_weather']}")
print(f"오늘의 기술 뉴스: {result_fixed_inputs['tech_news']}")